In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pandas as pd
import time
import sklearn
import tensorflow as tf

print(tf.__version__)

for module in np, sklearn, tf.keras, pd:
    print(module.__name__, module.__version__)

2.0.0
numpy 1.17.4
sklearn 0.20.0
tensorflow_core.keras 2.2.4-tf
pandas 0.25.3


In [5]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(housing.data, housing.target, random_state=7)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, random_state=11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scalered = scaler.fit_transform(x_train)
x_valid_scalered = scaler.transform(x_valid)
x_test_scalered = scaler.transform(x_test)
print(x_train_scalered.shape)
print(len(x_train_scalered.shape))
print(type(x_train_scalered))

(11610, 8)
2
<class 'numpy.ndarray'>


In [12]:
output_dir = 'generate_csv'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

def save_csv(output_dir, data, name_prefix, header=None, n_parts=10):
    path_format = os.path.join(output_dir, '{}_{:02d}.csv')
    filenames = []
    for file_idx, row_indices in enumerate(np.array_split(np.arange(len(data)), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filenames.append(part_csv)
        with open(part_csv, 'wt', encoding='utf-8') as f:
            if header is not None:
                f.write(header + '\n')
            for row_index in row_indices:
                f.write(','.join([repr(col) for col in data[row_index]]))
                f.write('\n')
    return filenames
                
train_data = np.c_[x_train_scalered, y_train]
valid_data = np.c_[x_valid_scalered, y_valid]
test_data = np.c_[x_test_scalered, y_test]
header_col = housing.feature_names + ['MidianHouseValue']
header_str = ','.join(header_col)
print(header_col)
print(header_str)

train_filenames = save_csv(output_dir, train_data, 'train', header_str, 20)
valid_filenames = save_csv(output_dir, valid_data, 'valid', header_str, 10)
test_filenames = save_csv(output_dir, test_data, 'test', header_str, 10)

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude', 'MidianHouseValue']
MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MidianHouseValue


In [16]:
filenames_dataset = tf.data.Dataset.list_files(train_filenames)
dataset = filenames_dataset.interleave(
    lambda x: tf.data.TextLineDataset(x).skip(1),
    cycle_length=5
)
for filename in filenames_dataset:
    print(filename)
for line in dataset.take(15):
    print(line.numpy())
    

tf.Tensor(b'generate_csv/train_19.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_12.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_15.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_11.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_05.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_13.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_10.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_02.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_17.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_08.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_09.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_07.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_03.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_18.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_00.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_04.csv', 